In [3]:
import numpy as np

class KnnBruteClassifier(object):
    def __init__(self, n_neighbors=1, weights='uniform', metric='l2'):
        self.n_neighbors = n_neighbors
        self.weights = weights
        self.metric = metric
     
    def fit(self, x, y):
        self.x_train = np.array(x)
        self.y_train = np.array(y)
    
    def predict(self, x):
        distances, indices = self.kneighbors(x, self.n_neighbors)
        weights = self.calculate_weights(distances)
        y_train_subset = self.y_train[indices]
        
        if self.weights == 'uniform':
            mode = np.array([np.argmax(np.bincount(row)) for row in y_train_subset])
        elif self.weights == 'distance':
            mode = np.array([np.argmax(np.bincount(row, weights=weights[i])) for i, row in enumerate(y_train_subset)])
        
        return mode
    
    def predict_proba(self, X):
        distances, indices = self.kneighbors(X, self.n_neighbors)
        weights = self.calculate_weights(distances)
        y_train_subset = self.y_train[indices]

        if self.weights == 'uniform':
            probabilities = np.array([np.bincount(row) / self.n_neighbors for row in y_train_subset])
        elif self.weights == 'distance':
            total_weights = np.sum(weights, axis=1)
            probabilities = np.array([np.bincount(row, weights=weights[i]) / total_weights[i] for i, row in enumerate(y_train_subset)])
        
        return probabilities
        
    def kneighbors(self, x, n_neighbors):
        distances = self.calculate_distances(x)
        indices = np.argsort(distances)[:, :n_neighbors]
        distances = np.take_along_axis(distances, indices, axis=1)

        return distances, indices
    
    def calculate_distances(self, x):
        if self.metric == 'l2':
            diff = x[:, :, np.newaxis] - self.x_train.T[np.newaxis, :, :]
            distances = np.sqrt(np.sum(diff ** 2, axis=1))        
        return distances
    
    def calculate_weights(self, distances):
        if self.weights == 'uniform':
            return np.ones(distances.shape)
        elif self.weights == 'distance':
            return 1 / distances